# Тестирование производительности

In [1]:
import time

def work(task):
    time.sleep(task)

## 1. Прямой замер времени

Данный вариант максимально просто и подходит для локального однократного временного замера времени. Важно понимать, что точность такого подхода весьма невысока.

In [2]:
import time

In [3]:
start = time.time()
work(2)
print(f'{time.time() - start} seconds')

2.000572681427002 seconds


Это не самый лучший таймер, есть варианты получше

In [12]:
start = time.perf_counter()
work(2)
print(f'User time: {time.perf_counter() - start} seconds')

User time: 2.0061835999949835 seconds


In [13]:
start = time.process_time()
work(2)
print(f'CPU time: {time.process_time() - start} seconds')

CPU time: 0.015625 seconds


## 2. `timeit`

Более удобный вариант с усредненным значением замера. Нужно учесть, что во время работы отключается сборщик мусора.

In [5]:
import timeit

N = 50

def step():
    work(0.01)

print(timeit.timeit(step, number=N) / N)
print([v / N for v in timeit.repeat(step, number=N, repeat=5)])

0.01530858000041917
[0.015373947999905794, 0.015408963999943808, 0.015442867999663577, 0.015406765999505296, 0.015420617999043316]


Если важно включение сборщика мусора, то 

In [6]:
timeit.timeit(step, setup="gc.enable()", number=N) / N

0.015198074000654743

При тестировании функций из текущего кода их нужно импортировать

In [7]:
timeit.timeit("work(0.01)", setup="from __main__ import work", number=N) / N

0.015283116000937298

In [8]:
# Так удобнее

timeit.timeit('work(0.01)', globals=globals(), number=N) / N

0.01525665600085631

### Командная строка

Помимо этого можно использовать данный модуль прямо из командной строки

```
python -m timeit -n 50 -s "from test import work" "work(0.001)"
```

In [9]:
!python -m timeit -n 50 -s "from test import work" "work(0.001)"

50 loops, best of 5: 15.1 msec per loop


## 3. Jupyter

Модуль выше можно использовать через [магические команды](https://ipython.readthedocs.io/en/stable/interactive/magics.html) Jupyter'а.

Можно замерить время конкретного выражения

In [11]:
%time work(0.5)

CPU times: total: 0 ns
Wall time: 512 ms


Или можно использовать `timeit`

In [19]:
%timeit -n20 -r2 work(0.1)

108 ms ± 63.9 µs per loop (mean ± std. dev. of 2 runs, 20 loops each)


Если использовать `%%`, то применяется к коду в ячейке

In [21]:
%%timeit -n20 -r2 

work(0.1)

108 ms ± 26.9 µs per loop (mean ± std. dev. of 2 runs, 20 loops each)


## 4. Профилирование

Все выше помогает замерить общее время работы части программы. Если перед вами стоит задача найти узкие места (задача оптимизации), то удобнее воспользоваться профилированием. В этом случае сильно страдает точность замера, но проще, понять, где проблема.

In [23]:
import cProfile

def work2():
    for _ in range(3):
        work(0.5)

cProfile.run('work2()')

         10 function calls in 1.521 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        3    0.000    0.000    1.521    0.507 2391014705.py:3(work)
        1    0.000    0.000    1.521    1.521 3803998991.py:3(work2)
        1    0.000    0.000    1.521    1.521 <string>:1(<module>)
        1    0.000    0.000    1.521    1.521 {built-in method builtins.exec}
        3    1.521    0.507    1.521    0.507 {built-in method time.sleep}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




* `ncalls` - количество вызовов
* `tottime` - время проведенное внутри функции (исключая все вызовы других функций)
* `cumtime` - полное время, проведенное в этой функции
* `percall` - время за один запрос

In [26]:
!python -m cProfile test.py

         5 function calls in 0.506 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.506    0.506 test.py:1(<module>)
        1    0.000    0.000    0.506    0.506 test.py:3(work)
        1    0.000    0.000    0.506    0.506 {built-in method builtins.exec}
        1    0.505    0.505    0.505    0.505 {built-in method time.sleep}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}


